In [34]:
import os
import json
import base64
import numpy as np
from io import BytesIO
from PIL import Image
import cv2


Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0001.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0002.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0003.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0004.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0005.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0006.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0007.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0008.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0009.jpg
Saved: /media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons/training0010.jpg


In [ ]:

# Paths
input_folder = "/media/arush/New Volume/Work/Giloma_Kaggle/Data/Glioma_MDC_2025_training/Glioma_MDC_2025_training"  # Folder with JSON files
output_folder = "/media/arush/New Volume/Work/Giloma_Kaggle/Data/created_image_with_polygons"  # Folder for output images


In [ ]:

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Iterate over all files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):  # Process only JSON files
        json_path = os.path.join(input_folder, file_name)
        
        # Read the JSON file
        with open(json_path, 'r') as file:
            data = json.load(file)
        

        # Extract Base64 image data
        image_data_base64 = data.get("imageData")
        if not image_data_base64:
            print(f"No imageData found in {file_name}")
            continue

        # Extract polygon points
        if "shapes" not in data or not data["shapes"]:
            print(f"No shapes found in {file_name}")
            continue

        # Decode the image data
        image_data = base64.b64decode(image_data_base64)
        image = Image.open(BytesIO(image_data))

        # Convert PIL image to OpenCV format (numpy array)
        image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        for shapes in data["shapes"]:

            polygon_points = shapes["points"]
            polygon_points = np.array(polygon_points, dtype=np.int32)

            if shapes["label"] == "Mitosis":
              # Draw the polygon on the image
              cv2.polylines(image_cv, [polygon_points], isClosed=True, color=(0, 0, 255), thickness=2)
            
            if shapes["label"] == "Non-mitosis":
              # Draw the polygon on the image
              cv2.polylines(image_cv, [polygon_points], isClosed=True, color=(0, 255, 0), thickness=2)
                 
        
        # Convert OpenCV image back to PIL format
        image_with_polygon = Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))
        
        # Save the image to the output folder
        output_image_path = os.path.join(output_folder, file_name.replace(".json", ".jpg"))
        image_with_polygon.save(output_image_path)
        print(f"Saved: {output_image_path}")
